In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#!pip install optuna

In [ ]:
#!pip install optuna-integration[xgboost]

In [ ]:
import math
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import joblib
from sklearn.model_selection import RandomizedSearchCV
# check xgboost version
from xgboost import XGBRegressor

#import optuna
#from optuna.integration import XGBoostPruningCallback

#torch stuff
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#from torch_geometric.utils import dense_to_sparse


import lightgbm as lgb
from lightgbm import LGBMRegressor
import holidays
us_holidays = holidays.country_holidays('US')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
#!pip install lime

In [ ]:

execfile('/content/drive/MyDrive/Thesis/models_training/model_training.py')

Using device: cpu


In [ ]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
suf = "_normal_xgb_11_08"

returns = pd.read_csv(f"{models_dir}/test_returns_predictions{suf}.csv").rename(
    columns={
        "end_station_cluster": "start_station_cluster",
        "pred": "returns"
        }
    )[['started_at_hourly', 'returns', 'start_station_cluster']]

In [ ]:
df = pd.read_csv(f'{data_dir}/final_model_input_partial_scale_3.csv')
df = df.merge(returns, on=['started_at_hourly', 'start_station_cluster'], how='left').fillna(0)

df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])
df = df.sort_values(by=['start_station_cluster', 'started_at_hourly'])

df_test = df[pd.to_datetime(
    df['started_at_hourly']) >= pd.to_datetime("2024-01-01 00:00:00")]
df_train = df[pd.to_datetime(
    df['started_at_hourly']) < pd.to_datetime("2024-01-01 00:00:00")]

del df

In [ ]:
target_scaler = joblib.load(f'{models_dir}/target_scaler.sav')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# GNN Variance Embedding

In [ ]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            'is_holiday',
            #'flag_added',
            'temp',
            'dwpt',
            'rhum',
            'prcp',
            'wdir',
            'wspd',
            'pres',
            'coco',
            "demand_lag_1_h",
            "demand_lag_2_h",
            "demand_lag_24_h",
            "temp_lag_1_h",
            "temp_lag_2_h",
            "temp_lag_24_h",
            "prcp_lag_1_h",
            "prcp_lag_2_h",
            "prcp_lag_24_h",
            "rhum_lag_1_h",
            "rhum_lag_2_h",
            "rhum_lag_24_h",
            "wspd_lag_1_h",
            "wspd_lag_2_h",
            "wspd_lag_24_h",
            "total_demand_1h",
            "demand_degrees_1h",
            'returns'
]
FEATURES.extend([f"dim_var_{i}" for i in range(50)])

## XGBoost

In [ ]:
@scope.define
def to_int(x):
    """
    Custom Hyperopt scope function to cast a value to an integer.

    :param x: Value to be cast to an integer.
    :type x: Any
    :return: Integer representation of the input value.
    :rtype: int
    """
    return int(x)


In [ ]:


model, model_preds = train_model(df_train, df_test, FEATURES, model='xgb', tuning=False)

overall
MSE: 3.5493534223843457
RMSE: 1.8839727764445924
MAE: 1.0002914774284006
MAPE: 0.39750226024213126

Non-zero
MSE: 7.2277160864076855
RMSE: 2.6884412001023352
MAE: 1.695786534866335
MAPE: 0.3557591269694381

Zeros
MSE: 0.5418079930888501
RMSE: 0.7360760783294414
MAE: 0.4316327637149571
MAPE: 0.4316327637149571


In [ ]:
# from lime.lime_tabular import LimeTabularExplainer
# explainer = LimeTabularExplainer(
#     training_data=df_train.values,  # Use training data to initialize
#     feature_names=FEATURES,  # Column names of your data
#     mode='regression'
# )

In [ ]:
#11_01
3.1785156823909997
1.7828392194449278
0.374173850198251

#11_08
3.063089630068344
1.7501684576258205
0.35523741143766596

#11-09 scaled
3.5493534223843457
1.8839727764445924
0.39750226024213126

#11-09 unscaled
3.6902228556581362
1.9209952773648706
0.41701083809631545

#11-28
3.5493534223843457
1.8839727764445924
0.39750226024213126

0.3949223304344264

In [ ]:
suf = "_k_xgb_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## LightGBM

In [ ]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='lgbm')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.427494 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15944
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 85
[LightGBM] [Info] Start training from score 1.005143
3.6305200729366023
1.905392367187557
0.4010697441810401


In [ ]:
#11_09
3.6510328057191144
1.9107675959464863
0.4032005166101049

#11-28
3.6305200729366023
1.905392367187557
0.4010697441810401

0.4032005166101049

In [ ]:
suf = "_k_lgbm_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## Random Forest

In [ ]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='rf')

overall
MSE: 3.804611499929171
RMSE: 1.9505413350988414
MAE: 1.0437195692220793
MAPE: 0.4303702703234751

Non-zero
MSE: 7.661438392962496
RMSE: 2.767930344673163
MAE: 1.7267541089471425
MAPE: 0.3632511667827063

Zeros
MSE: 0.6511480833994748
RMSE: 0.8069374718027877
MAE: 0.4852489681329114
MAPE: 0.4852489681329114


In [ ]:
#11_01
3.6042873699256828
1.8984960810930538
0.4084309933647605

#11_28
3.78203289464364
1.9447449433392647
0.4310134176146017

In [ ]:
suf = "_k_rf_11_28"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model